In [ ]:
import os
import re
import io
import sys
import glob
import enum
import json
import dask
import xlrd
import base64
import skimage
import imageio
import requests
import datetime
import tifffile
import numpy as np
import pandas as pd
import sqlalchemy as db

from scipy import ndimage
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 1

sys.path.append('../..')
%aimport opencell.imaging.images
%aimport opencell.imaging.managers
%aimport opencell.imaging.processors
%aimport opencell.database.operations

from opencell import constants, file_utils
from opencell.cli import database as db_cli
from opencell.cli import imaging as imaging_cli
from opencell.database import models
from opencell.database import operations as ops
from opencell.database import utils as db_utils
from opencell.imaging import utils as im_utils
from opencell.imaging import processors
from opencell.imaging.images import RawPipelineTIFF

In [ ]:
def plt_hist(vals, **kwargs):
    counts, edges = np.histogram(vals, **kwargs)
    plt.plot(edges[1:], counts)

In [ ]:
def profile_com(profile):
    profile = np.array(profile)
    p = profile - profile.mean()
    p[p < 0] = 0
    x = np.arange(len(p))
    com = (p*x).sum()/p.sum()
    return com

### Download the z-profiles for all FOVs from opencell database

In [ ]:
url = db_utils.url_from_credentials('../../db-credentials-cap.json')
engine = db.create_engine(url)
session_factory = db.orm.sessionmaker(bind=engine)
Session = db.orm.scoped_session(session_factory)

In [ ]:
# instantiate FOV processors from opencelldb
fovs = Session.query(models.MicroscopyFOV).all()
ps = [processors.FOVProcessor.from_database(fov) for fov in fovs]

In [ ]:
# construct FOV metadata (fov_id, plate_id, well_id, imaging_round_id)
rows = []
for p in ps:
    rows.append({
        'p': p,
        'fov_id': p.fov_id,
        'plate_id': p.plate_id,
        'well_id': p.well_id,
        'target_name': p.target_name,
        'step_size': p.z_step_size(),
    })

In [ ]:
fov_metadata = pd.DataFrame(data=rows)
fov_metadata.head()

In [ ]:
def all_results(kind):
    '''
    Aggregate results whose data column is a dict (not a list)
    '''
    results = Session.query(models.MicroscopyFOVResult)\
        .filter(models.MicroscopyFOVResult.kind == kind).all()  
    data = [{
        'fov_id': result.fov.id, 
        'line_id': result.fov.cell_line_id, 
        'pml_id': result.fov.dataset.pml_id,
        **result.data
    } for result in results]
    df = pd.DataFrame(data=data)
    return df

In [ ]:
# all z-profiles
fov_profiles = all_results('z-profiles')
fov_profiles.shape

In [ ]:
# all raw-tiff-metadata
raw_tiff_metadata = all_results('raw-tiff-metadata')
raw_tiff_metadata.shape

In [ ]:
raw_tiff_metadata = raw_tiff_metadata[[
    'fov_id', 'line_id', 'pml_id', 
    'laser_power_405_405', 'exposure_time_405', 
    'laser_power_488_488', 'exposure_time_488',
    'src_filepath'
]]

In [ ]:
# merge metadata and features on fov_id
data = pd.merge(fov_metadata, fov_profiles, left_on='fov_id', right_on='fov_id', how='inner')
data.shape

In [ ]:
# merge raw_tiff_metadata
data = pd.merge(data, raw_tiff_metadata, left_on='fov_id', right_on='fov_id', how='inner')

In [ ]:
data.to_csv('../../cache/2020-02-12-all-z-profiles.csv')

### Load from local cache

In [ ]:
# load the data from a local cache and parse the JSON columns
data = pd.read_csv('../../cache/2020-02-12-all-z-profiles.csv')
for ind, row in data.iterrows():
    try:
        data.at[ind, '405'] = json.loads(row['405'].replace('\'', '"'))
        data.at[ind, '488'] = json.loads(row['488'].replace('\'', '"'))
    except:
        pass

In [ ]:
data.head()

### Plot z-profiles

In [ ]:
data.loc[data.fov_id == 21475].src_filepath.values

In [ ]:
plt.plot(data.loc[data.fov_id == 21475].iloc[0]['488']['mean'])

In [ ]:
plt.plot(data.loc[data.fov_id == 21638].iloc[0]['488']['mean'])

In [ ]:
# max second derivative of the mean z-profile
data['max_dzdz'] = None
for ind, row in data.iterrows():
    try:
        data.at[ind, 'max_dzdz'] = (np.abs(np.diff(np.diff(row['488']['mean']))).max())
    except:
        pass

In [ ]:
d = data.loc[(data.pml_id_x == 'PML0225') & (data.max_dzdz > 0) & (data.exposure_time_488 > 1)]
d.shape

In [ ]:
offset = 99
nrows, ncols = 10, 10
fig, axs = plt.subplots(nrows, ncols, figsize=(24, 24))
count = offset
for ind in range(nrows):
    for jnd in range(ncols):
        if count == d.shape[0]:
            break
        ax = axs[ind][jnd]
        ax.plot(d.iloc[count]['488']['mean'])
        ax.set_title(d.iloc[count].fov_id)
        ax.set_xticks([])
        count += 1

In [ ]:
data.loc[data.max_dzdz > 100].groupby('pml_id_x').count().to_csv('max_dzdz.csv')

In [ ]:
d.src_filepath.values

### Distributions of cell layer center

In [ ]:
# calculate the center of mass of the mean intensity profile
data['com_405'] = None
data['com_488'] = None
for ind, row in data.iterrows():
    for channel in ['405', '488']:
        try:
            profile = row[channel]['mean']
            data.at[ind, 'com_%s' % channel] = profile_com(profile)
        except:
            print(ind)

data['com_405'] = data.com_405 * data.step_size
data['com_488'] = data.com_488 * data.step_size
data['delta'] = data.com_488 - data.com_405

In [ ]:
# calculate the total depth of each z-stack
data['depth_405'] = None
data['depth_405'] = None
for ind, row in data.iterrows():
    for channel in ['405', '488']:
        try:
            profile = row[channel]['mean']
            data.at[ind, 'depth_%s' % channel] = len(profile)
        except:
            print(ind)
data['depth_405'] = data.depth_405 * data.step_size
data['depth_488'] = data.depth_488 * data.step_size

In [ ]:
# the max of the p9999 z-profiles
data['max_405'] = None
data['max_488'] = None
for ind, row in data.iterrows():
    try:
        data.at[ind, 'max_405'] = max(row['405']['max'])
        data.at[ind, 'max_488'] = max(row['488']['max'])
    except:
        pass

In [ ]:
# distribution of distances of the cell layer center from the bottom of the stack
plt_hist(data.com_405[~data.com_405.isna()], bins=np.arange(0, 15, .2), density=True)
plt_hist(data.com_488[~data.com_488.isna()], bins=np.arange(0, 15, .2), density=True)

In [ ]:
# the percent of FOVs with a cell layer center close to the bottom of the stack
(data.com_405 < 5).sum() / data.shape[0]

In [ ]:
# distribution of distances from the cell layer center to the top of the stack
d = data.loc[~data.com_405.isna()]
plt_hist(d.depth_405 - d.com_405, bins=np.arange(0, 30, .2), density=True)

d = data.loc[~data.com_488.isna()]
plt_hist(data.depth_488 - data.com_488, bins=np.arange(0, 30, .2), density=True)

In [ ]:
# the set of unique stack depths
sorted(data.depth_405.unique())

In [ ]:
# distribution of maximum intensities
plt_hist(data.loc[~data.max_405.isna()].max_405, bins=np.arange(0, 3000, 30))
plt_hist(data.loc[~data.max_488.isna()].max_488, bins=np.arange(0, 65000, 1000))

In [ ]:
# the number of overexposed FOVs
data.loc[data.max_488 > 65000].shape

In [ ]:
# distribution of differences between 488 and 405 centers
plt_hist(data.delta, bins=np.arange(-3, 3, .1), density=True)

# the same distribution but only for FOVs with bright GFP
plt_hist(data.loc[data.max_488 > 15000].delta, bins=np.arange(-3, 3, .1), density=True)

In [ ]:
columns = ['plate_id', 'well_id', 'target_name', 'step_size', 'pml_id', 'com_405', 'com_488', 'delta']

In [ ]:
# inspect the offsets for all FOVs from a particular target
data_crop = data.loc[data.target_name.apply(lambda s: s.startswith('H1F'))][columns]
data_crop

In [ ]:
row = data.iloc[13908]
plt.plot(row['405']['mean'])
plt.plot(row['488']['mean'])

### Scatterplot of exposure and max intensities

In [ ]:
data['exposure_488'] = data.laser_power_488_488 * data.exposure_time_488

In [ ]:
_data = data.dropna(how='any', axis=0, subset=['exposure_488', 'max_488'])

In [ ]:
_ = plt.hist(_data.exposure_time_488[_data.exposure_time_488 < 500], bins=100)

In [ ]:
plt.figure(figsize=(12, 12))
h, xedges, yedges = np.histogram2d(
    _data.exposure_time_488,
    _data.max_488,
    bins=(np.arange(0, 550, 10), np.arange(0, 2**14, 1000)),
    density=False)

ax = plt.gca()
ax.pcolormesh(xedges, yedges, np.log(h.T + 1))
ax.set_xlim(xedges[0], xedges[-1])
ax.set_ylim(yedges[0], yedges[-1])

In [ ]:
# pml_ids with under-exposed FOVs
counts = _data.loc[(_data.exposure_time_488 < 200) & (_data.max_488 < 5000)].groupby(['pml_id_x', 'plate_id', 'target_name']).count().fov_id
counts.loc[counts > 2]

In [ ]:
_data.loc[_data.target_name == 'ELOVL5'].drop(labels=['405', '488'], axis=1)

In [ ]:
# pml_ids with under-exposed FOVs
_data.loc[
    (_data.exposure_time_488 < 200) & (_data.max_488 < 10000) & (_data.pml_id_x == 'PML0235')
].src_filepath.values

### Development of cell-layer-center-finding with cherrypicked example TIFFs

In [ ]:
for p in data.p:
    p.src_roots = {}

In [ ]:
step_size = row.step_size
src_filepath = row.p.src_filepath()
path = os.path.join('/Volumes/ml_group/PlateMicroscopy', src_filepath)

In [ ]:
# diffuse-ish nuclear GFP signal (this is PRKDC in D01 on P0014)
path = '/Users/keith.cheveralls/image-data/raw-pipeline-microscopy/PML0234/raw_data/MMStack_165-B5-21.ome.tif'

# PCNA and PTMA: nucleus-associated targets with 0.5um step size
# path = '/Users/keith.cheveralls/image-data/H6_9_PCNA.ome.tif'
# path = '/Users/keith.cheveralls/image-data/E1_15_PTMA.ome.tif'

In [ ]:
tiff = RawPipelineTIFF(path)
tiff.parse_micromanager_metadata()
tiff.validate_micromanager_metadata()
tiff.split_channels()

In [ ]:
# cell layer center using Hoechst
cen, profile_405 = tiff.find_cell_layer(tiff.stacks['405'])
cen

In [ ]:
# cell layer center using GFP
cen, profile_488 = tiff.find_cell_layer(tiff.stacks['488'])
cen

In [ ]:
# we observe a four-slice offset in the cell layer center between hoechst and GFP
# this corresponds to 0.8um, which is more than what christian measured (which was 0.5um)
plt.plot((profile_405[:] - profile_405.min())/(profile_405.max() - profile_405.min()))
plt.plot((profile_488[3:] - profile_488.min())/(profile_488.max()- profile_488.min()))

In [ ]:
stacks, result = tiff.crop_and_align_cell_layer(-5, 6, .5)
result

In [ ]:
sz = 155
plt.figure(figsize=(18,18))
plt.imshow(np.concatenate((stack[10, :sz, :sz], stack[25, :sz, :sz]), axis=1))

In [ ]:
plt.figure(figsize=(18, 18))
im405 = im_utils.autoscale(stacks['405'][:, :, :100].mean(axis=2), p=1)
im488 = im_utils.autoscale(stacks['488'][:, :, :100].mean(axis=2), p=0)
imboth = np.concatenate((im405[:,:,None], im488[:,:,None]), axis=2).max(axis=2)
plt.imshow(np.concatenate((imboth[:,:,None], im488[:,:,None], im488[:,:,None]), axis=2))

### Testing ndimage.shift

In [ ]:
# testing ndimage.shift
# note that lower interpolation orders (below 3) result in some smoothing of the shifted image,
# because the shifted slices are weighted means of the original slices
im = tiff.stacks[tiff.laser_405][:, :crop, :crop]
out = ndimage.shift(im, (.1, 0, 0), order=1)

In [ ]:
crop = 111
plt.figure(figsize=(16, 12))
plt.imshow(np.concatenate((im[33, :crop, :crop], out[33, :crop, :crop]), axis=1))

In [ ]:
# compare mean of two slices to original slice (e.g., shifting by 0.5 with order=1)
plt.figure(figsize=(16, 12))
plt.imshow(np.concatenate((im[33, :crop, :crop], im[33:35, :crop, :crop].mean(axis=0)), axis=1))

In [ ]:
# compare 2x downsampled images to original images
plt.figure(figsize=(16, 12))
imin = im[:100, :110, :110]
imout = skimage.transform.downscale_local_mean(imin, (1, 2, 2))
imout = skimage.transform.rescale(imout, scale=(1, 2, 2), order=0, multichannel=False)
plt.imshow(np.concatenate((imin[33, :crop, :crop], imout[33, :crop, :crop]), axis=1))

### Refactoring nathan's method to select in-focus stacks

In [ ]:
# a raw stack
stack = tifffile.imread('/Users/keith.cheveralls/image-data/MMStack_10-B9-10.ome.tif')
dapi_stack = stack[:131, :, :]
stack.shape

In [ ]:
dapi_stack.max(axis=1).shape

In [ ]:
viz.imshow(dapi_stack.max(axis=2))

In [ ]:
# blur_vals = np.array([cv2.Laplacian(zslice, cv2.CV_64F).var() for zslice in dapi_stack])
sum_vals = np.array([zslice.mean() for zslice in dapi_stack]).astype(float)
plt.plot(sum_vals)

In [ ]:
# suppose one z-slice is underexposed by a factor of two
# sum_vals[30] = sum_vals[30]/10
dist = sum_vals - sum_vals.mean()
dist[dist < 0] = 0
dist /= dist.sum()
plt.plot(dist * 30)
plt.plot(np.cumsum(dist))
np.argwhere(np.cumsum(dist) > .5).min()

In [ ]:
# check derivative for spikes due to isolated unexposed z-slices
np.abs(np.diff(sum_vals)).max()

In [ ]:
# calculate the mean and variance of the intensity profile in z
sum_vals -= sum_vals.min()
sum_vals /= sum_vals.sum()
x = np.arange(len(sum_vals))
xm = (x * sum_vals).sum()
xv = (x * x * sum_vals).sum()
xs = np.sqrt(xv - xm**2)
xm, xs

In [ ]:
xm - 2*xs, xm + 2*xs